In [ ]:
!pip install mne
!pip install tqdm

In [ ]:
import random
import os
import pandas as pd
from tqdm.notebook import tqdm
from mne.datasets import eegbci
from mne.io import read_raw_edf
from mne import events_from_annotations, Epochs

In [ ]:
# Set seed and split subjects
random.seed(42)
subject_ids = list(range(1, 110))
random.shuffle(subject_ids)

n_total = len(subject_ids)
n_train = int(n_total * 0.7)
n_val = int(n_total * 0.15)
n_test = n_total - n_train - n_val

train_subjects = subject_ids[:n_train]
val_subjects = subject_ids[n_train:n_train+n_val]
test_subjects = subject_ids[n_train+n_val:]

assert(len(train_subjects)+len(val_subjects)+len(test_subjects) == 109)

# EEG settings
runs = [4, 6, 8, 10, 12, 14]
left_right = [4, 8, 12]
both_hf = [6, 10, 14]

T_MIN = 0
T_MAX = 4
LOW_BAND = 7
HIGH_BAND = 30

In [ ]:
# function to load user run information
def load_user_run(subject_id, run):
    file_paths = eegbci.load_data(subjects=subject_id, runs=run, verbose=False)
    raw_list = [read_raw_edf(f, preload=True, verbose=False) for f in file_paths]
    return raw_list[0]

In [ ]:
# function to get the proper df information
def process_subject(subject_id, split_name, epoch_time):
    subject_df = pd.DataFrame(columns=['subject_id', 'run', 'time_index', 'augmentation', 'X', 'label'])
    for run in runs:
        try:
            raw = load_user_run(subject_id, run)
            raw.filter(l_freq=LOW_BAND, h_freq=HIGH_BAND, fir_design='firwin', verbose=False)
            events, event_id = events_from_annotations(raw)
            epochs = Epochs(raw, events, event_id=event_id, tmin=T_MIN, tmax=epoch_time, baseline=None, preload=True)


            # sets proper labels
            label_map = {
                1: 'rest',
                2: 'left_hand' if run in left_right else 'both_hands',
                3: 'right_hand' if run in left_right else 'both_feet'
            }
            labels = [label_map[e] for e in epochs.events[:, 2]]

            # epochs data
            for i, epoch in enumerate(epochs.get_data()):
                subject_df = pd.concat([subject_df, pd.DataFrame([{
                    'subject_id': subject_id,
                    'run': run,
                    'time_index': i * epoch_time,
                    'augmentation': 'none',
                    'X': epoch,
                    'label': labels[i],
                }])], ignore_index=True)
        except Exception as e:
            print(f"Subject {subject_id} (run {run}) failed: {e}")
    return subject_df


In [ ]:
# mount to google drive
from google.colab import drive
drive.mount('/content/drive')
# Main loop
drive_path = "/content/drive/MyDrive/eeg_data"
os.makedirs(drive_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
splits = {
    'train': train_subjects,
    'val': val_subjects,
    'test': test_subjects,
}

In [ ]:
EPOCH_DURATION = 4
for split, subjects in splits.items():
    all_df = pd.DataFrame()
    for sid in tqdm(subjects, desc=f"Processing {split} subjects"):
        df = process_subject(sid, split, EPOCH_DURATION)
        all_df = pd.concat([all_df, df], ignore_index=True)
    all_df.to_pickle(f"{drive_path}/eegmmidb_{split}_df.pkl")
    print(f"{split} set saved with {len(all_df)} samples.")

print("Finished")

Processing train subjects:   0%|          | 0/76 [00:00<?, ?it/s]

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 p

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


...

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped


Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 641 original time points ...
0 bad epochs dropped
test set saved with 3054 samples.
Finished
